# Import Libraries

In [96]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import plotly.express as px
import plotly.io as pio
import plotly
from ydata_profiling import  ProfileReport
import PyQt5 as qt
from IPython.display import display, Markdown
#Enable graphing inside jupytor
#pip install PyQt5 #Install it if not installed
get_ipython().run_line_magic('matplotlib', 'inline')
matplotlib.get_backend()

'module://matplotlib_inline.backend_inline'

# Load Data

In [97]:
df = pd.read_excel('maintenance_cleaned_extended.xlsx')

# Visualization Functions

## Bar, Scatter, Line charts

In [98]:
def myPlot1(data, xs, ys, clr, plotType, title, sort_by=None, ascending=True):
    if sort_by is not None:
        data_sorted = data.sort_values(by=sort_by, ascending=ascending)
    else:
        data_sorted = data
    xt = str(xs)
    yt = str(ys)
    xs = data_sorted[xs]
    ys = data_sorted[ys]
    clrt=str(clr)
    clr = data_sorted[clr].astype(str) if clr else None
    if plotType == 'bar':
        fig = px.bar(x=xs, y=ys, color=clr, title=title + ' Analysis')
    elif plotType == 'scatter':
        fig = px.scatter(x=xs, y=ys, color=clr, title=title + ' Analysis')
    elif plotType == 'line':
        fig = px.line(x=xs, y=ys, color=clr, title=title + ' Analysis')
    fig.update_layout(title_x=0.0)
    fig.update_layout(xaxis_title=xt, yaxis_title=yt)
    fig.update_layout(legend_title_text=clrt)
    fig.show()

In [99]:
def myPlot11(data, xs, ys, clr, plotType, title, sort_by=None, ascending=True):
    if sort_by is not None:
        data_sorted = data.sort_values(by=sort_by, ascending=ascending)
    else:
        data_sorted = data
    xt = str(xs)
    yt = str(ys)
    xs = data_sorted[xs]
    ys = data_sorted[ys]
    #clr = data_sorted[clr].astype(str) if clr else None
    if plotType == 'bar':
        fig = px.bar(x=xs, y=ys, title=title + ' Analysis')
        fig.update_traces(marker_color=clr)  # Bar-specific color update
    elif plotType == 'scatter':
        fig = px.scatter(x=xs, y=ys, title=title + ' Analysis')
        fig.update_traces(marker=dict(color=clr))  # Scatter-specific color update
    elif plotType == 'line':
        fig = px.line(x=xs, y=ys, title=title + ' Analysis')
        fig.update_traces(marker_color=clr)  # Bar-specific color update
    fig.update_layout(title_x=0.0)
    fig.update_layout(xaxis_title=xt, yaxis_title=yt)
    fig.show()

## Sunburst chart

In [100]:
def mySunBurst(data, name1,name2, value,clr, title):
    fig = px.sunburst(
        data_frame=data,
        path=[name1,name2],   # Add both cost_category and damage type to the hierarchy
        #path=name,
        values=value,  # Define the values (damage_count)
        color=clr,
        title=title +' Analysis'
    )
    fig.update_layout(title_x=0.0)
    fig.update_layout(height=600, margin=dict(t=50, l=25, r=25, b=25))
    fig.show()

In [101]:
def TopTailCount(data,count,groupField,SumField):
    # Get top & Tail COUNT records based on total groupFiled
    top = data.groupby(groupField)[SumField].sum().sort_index(ascending=False).nlargest(count).index
    tail = data.groupby(groupField)[SumField].sum().sort_index(ascending=False).nsmallest(count).index
    # Filter for top corporates
    top_filtered_data = data[data[groupField].isin(top)].sort_index(ascending=False)
    tail_filtered_data = data[data[groupField].isin(tail)].sort_index(ascending=False)
    return top_filtered_data, tail_filtered_data

In [102]:
def coloring(data,top,tail):
    colors = [assign_color(index,top,tail) for index in data.index]
    return colors

In [103]:
def sumOfsum(data,groupField1,groupField2,sumField):
    # Step 1: Calculate total sum of CorpLocCostSum for each corporate
    df_field1_sum = data.groupby(groupField1)[sumField].sum().reset_index(name='First Sum')
    # Step 2: Merge the total sum back into the original DataFrame
    df_field2_sum = data.groupby([groupField1, groupField2])[sumField].sum().reset_index(name='Total Sum')
    # Step 3: Merge the total sum of each corporate into this grouped data
    df_field2_sum = df_field2_sum.merge(df_field1_sum, on=groupField1)
    # Step 4: Sort by the total corporate cost (TotalCorpCost) and within that by CorpLocCostSum
    df_field2_sum = df_field2_sum.sort_values(by=['First Sum', 'Total Sum'], ascending=False)
    # Step 5: Drop the 'TotalCorpCost' column if you no longer need it
    df_field2_sum = df_field2_sum.drop(columns=['First Sum'])
    return df_field2_sum

## Bi - Variance Analysis

### Corporate Client Profitability Insights

### Corporate Client Profitability

In [104]:
text="Our analysis indicates that customers vary significantly in their profitability. This information should guide our decisions about which branches to keep, close, or expand based on their ability to generate sustainable revenue."
display(Markdown('#### *'+text+'*'))

df_corporate = df.groupby('corporate')['cost'].sum().reset_index(name='Corporate Totla Service Cost').sort_values(by='Corporate Totla Service Cost',ascending=False)
top_filterd_data, tail_filtered_data = TopTailCount(df_corporate,3,'corporate','Corporate Totla Service Cost')
colors = coloring(df_corporate,top_filterd_data, tail_filtered_data)
myPlot11(df_corporate,'corporate','Corporate Totla Service Cost',colors,'bar','Corporate Client Profitability', sort_by='Corporate Totla Service Cost', ascending=False)


#### *Our analysis indicates that customers vary significantly in their profitability. This information should guide our decisions about which branches to keep, close, or expand based on their ability to generate sustainable revenue.*

#### Corporate Client Profitability across Service Locations

In [105]:
text = "Analysis reveals significant variations in profitability across different client companies. This suggests a need to focus on branches serving the largest and most profitable clients, while evaluating the performance of other branches to improve their efficiency and increase their overall contribution to profits"
display(Markdown('#### *'+text+'*'))
corporate_Location = sumOfsum(df,'corporate','location','cost')
myPlot1(corporate_Location,'corporate','Total Sum','location','bar','Corporate Client Profitability across Service Locations', sort_by=None, ascending=True)

text = "Analysis reveals significant variations in profitability across different client companies. This suggests a need to focus on branches serving the largest and most profitable clients, while evaluating the performance of other branches to improve their efficiency and increase their overall contribution to profits"
display(Markdown('#### *'+text+'*'))
mySunBurst(corporate_Location, 'corporate','location', 'Total Sum', 'Total Sum', 'Corporate Client Profitability across Service Locations')

#### *Analysis reveals significant variations in profitability across different client companies. This suggests a need to focus on branches serving the largest and most profitable clients, while evaluating the performance of other branches to improve their efficiency and increase their overall contribution to profits*

#### *Analysis reveals significant variations in profitability across different client companies. This suggests a need to focus on branches serving the largest and most profitable clients, while evaluating the performance of other branches to improve their efficiency and increase their overall contribution to profits*

#### Corporate Top COUNT Clients Profitability 

In [106]:
text="The chart visually represents the magnitude of the profit contribution from each of the major companies to the company's total profits. It can be observed that company 'Xe' is the largest contributor to profits, followed by 'Jordan Transports' and then 'Vestas'."
display(Markdown('#### *'+text+'*'))
top_filtered_data, tail_filtered_data = TopTailCount(df,5,'corporate','cost')
myData = top_filtered_data.groupby('corporate')['cost'].sum().reset_index(name='Corporate Cost Sum').sort_values('Corporate Cost Sum',ascending=False)
myPlot1(myData,'corporate','Corporate Cost Sum',None,'bar','Corporate Top Clients Profitability', sort_by=None, ascending=True)

#### *The chart visually represents the magnitude of the profit contribution from each of the major companies to the company's total profits. It can be observed that company 'Xe' is the largest contributor to profits, followed by 'Jordan Transports' and then 'Vestas'.*

#### Top Corporates and Their Service Locations' Income Distribution

In [107]:
text="This chart provides a breakdown of the total revenue generated by our top clients, analyzing the contributions from each of our service locations."
display(Markdown('#### *'+text+'*'))
top_filtered_data, tail_filtered_data = TopTailCount(df,5,'corporate','cost')
myData = sumOfsum(top_filtered_data,'corporate','location','cost')
myPlot1(myData,'corporate','Total Sum','location','bar',"Top Corporates and Their Service Locations' Income Distribution", sort_by=None, ascending=True)

#### *This chart provides a breakdown of the total revenue generated by our top clients, analyzing the contributions from each of our service locations.*

#### Top Corporates and Their Service Locations' Income Distribution

In [108]:
text="This chart provides a breakdown of the total revenue generated by our top clients, analyzing the contributions from each of our service locations."
display(Markdown('#### *'+text+'*'))
top_filtered_data, tail_filtered_data = TopTailCount(df,5,'corporate','cost')
myData = sumOfsum(top_filtered_data,'corporate','location','cost')
mySunBurst(myData, 'corporate','location', 'Total Sum', 'Total Sum', "Top Corporates and Their Service Locations' Income Distribution")

#### *This chart provides a breakdown of the total revenue generated by our top clients, analyzing the contributions from each of our service locations.*

### Service Location Profitability

#### Top 5 Corporates and Their Top 5 Service Locations' Income Distribution

In [109]:
text="The chart illustrates the income distribution for the top 5 companies and their top 5 service locations. 'Xe' stands out with the highest revenue, supported by locations such as 'Al-Azazi' and 'Al-Sharqiya'. Some companies may share the same service locations or benefit from different ones."
display(Markdown('#### *'+text+'*'))

corporate_Location = sumOfsum(df,'corporate','location','cost')
top_5_corporates = df.groupby('corporate')['cost'].sum().sort_index(ascending=False).nlargest(5).index
# Step 1: Filter for the top 5 corporates
filtered_data_top_corporates = corporate_Location[corporate_Location['corporate'].isin(top_5_corporates)]
# Step 2: For each corporate, find the top 5 service locations based on income
top_profitable_locations_per_corporate = pd.DataFrame()
for corporate in top_5_corporates:
    # Get the top 5 locations for this corporate
    top_locations = (filtered_data_top_corporates[filtered_data_top_corporates['corporate'] == corporate]
                     .nlargest(5, 'Total Sum'))
    # Identify the less profitable locations for this corporate (those not in the top 5)
    all_locations_for_corporate = filtered_data_top_corporates[filtered_data_top_corporates['corporate'] == corporate]
    top_profitable_locations = all_locations_for_corporate[all_locations_for_corporate['location'].isin(top_locations['location'])]
    # Append the less profitable locations to the  less DataFrame
    top_profitable_locations_per_corporate = pd.concat([top_profitable_locations_per_corporate, top_profitable_locations])
myPlot1(top_profitable_locations_per_corporate,'corporate','Total Sum','location','bar',"Top 5 Corporates and Their Top 5 Service Locations' Income Distribution", sort_by=None, ascending=True)

text="The chart illustrates the income distribution for the top 5 companies and their top 5 service locations. 'Xe' stands out with the highest revenue, supported by locations such as 'Al-Azazi' and 'Al-Sharqiya'. Some companies may share the same service locations or benefit from different ones."
display(Markdown('#### *'+text+'*'))
mySunBurst(top_profitable_locations_per_corporate,'corporate','location','Total Sum','Total Sum',"Top 5 Corporates and Their Top 5 Service Locations' Income Distribution")

#### *The chart illustrates the income distribution for the top 5 companies and their top 5 service locations. 'Xe' stands out with the highest revenue, supported by locations such as 'Al-Azazi' and 'Al-Sharqiya'. Some companies may share the same service locations or benefit from different ones.*

#### *The chart illustrates the income distribution for the top 5 companies and their top 5 service locations. 'Xe' stands out with the highest revenue, supported by locations such as 'Al-Azazi' and 'Al-Sharqiya'. Some companies may share the same service locations or benefit from different ones.*

#### Less Profitable Corporates (Not in Top 5)

In [110]:
text="The chart illustrates the income distribution for the less profitable companies, these should be encoureged through different ways to maximize their dealing with Ahmad Company ."
display(Markdown('#### *'+text+'*'))
print('Total Corporates =',df['corporate'].nunique(), 'Less Profitable Corporates (Not in Top 5)=',df['corporate'].nunique() - 5)
# Filter Corporates, get the tail corporates base on cost sum
top_filtered_data, tail_filtered_data = TopTailCount(df,50,'corporate','cost')
# Get the dataframe for these tail corporates
myData = tail_filtered_data.groupby('corporate')['cost'].sum().reset_index(name='Total Sum').sort_values(by='Total Sum',ascending=False)
# Plot the chart
myPlot1(myData,'corporate','Total Sum',None,'bar',"Less Profitable Corporates (Not in Top 5)", sort_by=None, ascending=True)


#### *The chart illustrates the income distribution for the less profitable companies, these should be encoureged through different ways to maximize their dealing with Ahmad Company .*

Total Corporates = 55 Less Profitable Corporates (Not in Top 5)= 50


#### Less Profitable Service Locations for Each Top 5 Corporate

In [111]:
corporate_Location = sumOfsum(df,'corporate','location','cost')
top_5_corporates = df.groupby('corporate')['cost'].sum().sort_index(ascending=False).nlargest(5).index
# Step 1: Filter for the top 5 corporates
filtered_data_top_corporates = corporate_Location[corporate_Location['corporate'].isin(top_5_corporates)]

# Step 2: For each corporate, find the top 5 service locations based on income
less_profitable_locations_per_corporate = pd.DataFrame()
top_profitable_locations_per_corporate = pd.DataFrame()

for corporate in top_5_corporates:
    # Get the top 5 locations for this corporate
    top_locations = (filtered_data_top_corporates[filtered_data_top_corporates['corporate'] == corporate]
                     .nlargest(5, 'Total Sum'))
    # Identify the less profitable locations for this corporate (those not in the top 5)
    all_locations_for_corporate = filtered_data_top_corporates[filtered_data_top_corporates['corporate'] == corporate]
    less_profitable_locations = all_locations_for_corporate[~all_locations_for_corporate['location'].isin(top_locations['location'])]
    top_profitable_locations = all_locations_for_corporate[all_locations_for_corporate['location'].isin(top_locations['location'])]
    # Append the less profitable locations to the  less DataFrame
    less_profitable_locations_per_corporate = pd.concat([less_profitable_locations_per_corporate, less_profitable_locations])
    top_profitable_locations_per_corporate = pd.concat([top_profitable_locations_per_corporate, top_profitable_locations])

# Display the less profitable service locations
#print("Less profitable service locations (not in top 5 for each corporate):")
#print(less_profitable_locations_per_corporate[['corporate', 'location', 'Total Sum']].sort_values(by='Total Sum'))

# Display the top profitable service locations
#print("top profitable service locations (in top 5 for each corporate):")
#print(top_profitable_locations_per_corporate[['corporate', 'location', 'Total Sum']].sort_values(by='Total Sum'))

text="The chart illustrates the income distribution for the less profitable locations, these should investgated, opertional cost vs income, if the operational cost is more than income, then they should be closed immediatly, employees can be shifted to other profitable locations, else their customers should be encoureged through different ways to maximize their dealing with these locations, hence, with Ahmad Company through hot discount promotions."
display(Markdown('#### *'+text+'*'))
myPlot1(less_profitable_locations_per_corporate,'location','Total Sum','corporate','bar',"Less Profitable Service Locations (Not in Top 5 for Each Corporate)", sort_by=None, ascending=True)

text="The chart illustrates the income distribution for the top profitable locations, employees of these locations should rewarded."
display(Markdown('#### *'+text+'*'))
myPlot1(top_profitable_locations_per_corporate,'location','Total Sum','corporate','bar',"Top Profitable Service Locations ( in Top 5 for Each Corporate)", sort_by=None, ascending=True)


#### *The chart illustrates the income distribution for the less profitable locations, these should investgated, opertional cost vs income, if the operational cost is more than income, then they should be closed immediatly, employees can be shifted to other profitable locations, else their customers should be encoureged through different ways to maximize their dealing with these locations, hence, with Ahmad Company through hot discount promotions.*

#### *The chart illustrates the income distribution for the top profitable locations, employees of these locations should rewarded.*